Sekcja 12 - Google Analytics Attribution Analysis

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


In [ ]:
%%bigquery --project august-bucksaw-330517

SELECT * FROM `jrjames83-1171.sampledata.conversion_paths` 
limit 5

,path,conversions,revenue
0,(not set) / (not set),2453.0,2140.06
1,google / organic,981.0,127.65
2,google / cpc,592.0,259.00
3,mall.googleplex.com / referral,545.0,0.00
4,dfa / cpm,160.0,0.00


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select split('google / cpc > google / cpc > brand / email', '>') as path
)
select 
lower(trim(path_row)) as row, 
index, 
array_length(path) as nbr_in_path

from base_table, unnest(path) as path_row with offset as index

,row,index,nbr_in_path
0,google / cpc,0,3
1,google / cpc,1,3
2,brand / email,2,3


In [ ]:
%%bigquery --project august-bucksaw-330517

with base_table as (
    select split(path, '>') as path
    from `jrjames83-1171.sampledata.conversion_paths` 
    where array_length(split(path, '>')) > 1
),

path_row_table as (
select 
lower(trim(path_row)) as row, 
index, 
array_length(path) as nbr_in_path
from base_table, unnest(path) as path_row with offset as index
)

select *, 
case when index = 0 then 'first_touch'
when nbr_in_path - 1 = index then 'last_touch'
else 'middle_touch'
end as touch_type
from path_row_table
limit 10

,row,index,nbr_in_path,touch_type
0,google / cpc,0,2,first_touch
1,google / cpc,1,2,last_touch
2,google / organic,0,2,first_touch
3,mall.googleplex.com / referral,1,2,last_touch
4,google / organic,0,2,first_touch
5,(not set) / (not set),1,2,last_touch
6,(not set) / (not set),0,2,first_touch
7,google / cpc,1,2,last_touch
8,google / cpc,0,2,first_touch
9,(not set) / (not set),1,2,last_touch
